# Practical Aspects of Image Classification

- **Topics will be covered in this chapter:**
    - Generating CAMs
    - Understanding the impact of batch normalization and data augmentation
    - Practical aspects to take care of during model implementation